In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import svm
from sklearn.semi_supervised import label_propagation
from sklearn.semi_supervised import LabelSpreading
import pandas as pd

rng = np.random.RandomState(0)

In [2]:
metro_data = pd.read_csv('metro_summary_data.csv')
sorted_metro_data = metro_data.sort_values('PercentTech', ascending=False)

In [3]:
sorted_metro_data

,Unnamed: 0,CBSA,TotalPeople,TotalTech,MedianAge,PercentHighSchool,PercentCollege,PercentPostGrad,MedianIncome,PercentFemale,PercentBornOutOfState,PercentImmigrant,Percent18to35,PercentTech,MedianRent,MedianHouseValue,DiversityIndex
204,205,"San Jose-Sunnyvale-Santa Clara, CA",1916862,114373,37.000000,0.677346,0.353995,0.160663,30000.000000,0.496868,0.546244,0.400493,0.255370,5.966679,1800.000000,800000.000000,0.701149
131,132,"Madison, WI",523793,20891,35.000000,0.757956,0.369048,0.129826,30000.000000,0.502044,0.409666,0.104973,0.299702,3.988408,850.000000,230000.000000,0.344415
26,27,"Bloomington, IL",173828,6648,33.000000,0.747929,0.292554,0.098655,25000.000000,0.511373,0.257571,0.059944,0.325857,3.824470,700.000000,165000.000000,0.347239
17,18,"Austin-Round Rock, TX",1881313,68831,34.000000,0.684594,0.302374,0.101011,27200.000000,0.499367,0.466017,0.170215,0.293115,3.658668,950.000000,240000.000000,0.609035
185,186,"Raleigh, NC",1209514,42795,36.000000,0.678530,0.312101,0.109033,27000.000000,0.511799,0.533912,0.136046,0.249252,3.538198,830.000000,225000.000000,0.565302
203,204,"San Francisco-Oakland-Hayward, CA",4656199,164574,38.000000,0.707027,0.354305,0.136603,30000.000000,0.507558,0.515518,0.324769,0.255497,3.534514,1500.000000,700000.000000,0.715049
211,212,"Seattle-Tacoma-Bellevue, WA",3735393,126339,37.000000,0.719753,0.297356,0.103938,30000.000000,0.500339,0.551715,0.200753,0.263899,3.382214,1100.000000,350000.000000,0.543487
230,231,"Trenton, NJ",370588,12258,38.000000,0.689709,0.271919,0.131127,23503.174603,0.506789,0.479295,0.245653,0.249377,3.307716,1000.000000,275000.000000,0.661787
239,240,"Washington-Arlington-Alexandria, DC-VA-MD-WV",3780523,121278,36.000000,0.701644,0.368529,0.177054,35000.000000,0.516928,0.701891,0.249536,0.272231,3.207969,1400.000000,380000.000000,0.703098
36,37,"Cedar Rapids, IA",220052,6407,37.000000,0.728519,0.221230,0.067529,27874.691358,0.509143,0.262711,0.046366,0.248941,2.911585,550.000000,145000.000000,0.229208


In [4]:
sorted_metro_data['LogMedianRent'] = np.log(sorted_metro_data['MedianRent'])
sorted_metro_data['LogMedianHouseValue'] = np.log(sorted_metro_data['MedianHouseValue'])
sorted_metro_data['LogTotalPeople'] = np.log(sorted_metro_data['TotalPeople'])

In [5]:
array2 = sorted_metro_data.as_matrix(['PercentCollege', 'PercentFemale'])

test_labels = [-1, -1, -1, 1, 1, 1, 1, 1, 1]
rng.shuffle(test_labels)

end_labels = [-1] * (array2.shape[0] - 15) + [0] * 6
rng.shuffle(end_labels)


labels2 = np.asarray(test_labels + end_labels)

prop2 = label_propagation.LabelPropagation(kernel='knn', n_neighbors=7, max_iter=100, alpha=1).fit(array2, labels2)
model_results = prop2.predict(array2)
#np.where(np.asarray(prop2[1]) > 0)
model_results

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [6]:
indices = np.where(np.asarray(model_results) > 0)
sorted_metro_data.iloc[indices]['CBSA']

Series([], Name: CBSA, dtype: object)

In [7]:
sorted_metro_data.head()

,Unnamed: 0,CBSA,TotalPeople,TotalTech,MedianAge,PercentHighSchool,PercentCollege,PercentPostGrad,MedianIncome,PercentFemale,PercentBornOutOfState,PercentImmigrant,Percent18to35,PercentTech,MedianRent,MedianHouseValue,DiversityIndex,LogMedianRent,LogMedianHouseValue,LogTotalPeople
204,205,"San Jose-Sunnyvale-Santa Clara, CA",1916862,114373,37.0,0.677346,0.353995,0.160663,30000.0,0.496868,0.546244,0.400493,0.255370,5.966679,1800.0,800000.0,0.701149,7.495542,13.592367,14.466200
131,132,"Madison, WI",523793,20891,35.0,0.757956,0.369048,0.129826,30000.0,0.502044,0.409666,0.104973,0.299702,3.988408,850.0,230000.0,0.344415,6.745236,12.345835,13.168852
26,27,"Bloomington, IL",173828,6648,33.0,0.747929,0.292554,0.098655,25000.0,0.511373,0.257571,0.059944,0.325857,3.824470,700.0,165000.0,0.347239,6.551080,12.013701,12.065822
17,18,"Austin-Round Rock, TX",1881313,68831,34.0,0.684594,0.302374,0.101011,27200.0,0.499367,0.466017,0.170215,0.293115,3.658668,950.0,240000.0,0.609035,6.856462,12.388394,14.447480
185,186,"Raleigh, NC",1209514,42795,36.0,0.678530,0.312101,0.109033,27000.0,0.511799,0.533912,0.136046,0.249252,3.538198,830.0,225000.0,0.565302,6.721426,12.323856,14.005729


In [8]:
sorted_metro_data.index



Int64Index([204, 131,  26,  17, 185, 203, 211, 230, 239,  36,
            ...
            168, 250,  50, 115,  60, 136,  32, 125,  62, 132],
           dtype='int64', length=251)

In [9]:
from sklearn import preprocessing

subset = sorted_metro_data[['PercentCollege',
                  'Percent18to35', 'LogMedianRent', 'DiversityIndex']]
nsmd = pd.DataFrame(
    index = subset.index,
    data = preprocessing.normalize(subset, axis=0),
    columns = subset.columns
)

plt.hist(nsmd.LogMedianRent)


(array([  4.,  28.,  48.,  68.,  50.,  26.,  10.,  10.,   6.,   1.]),
 array([ 0.05748738,  0.05893052,  0.06037367,  0.06181681,  0.06325995,
         0.0647031 ,  0.06614624,  0.06758939,  0.06903253,  0.07047567,
         0.07191882]),
 <a list of 10 Patch objects>)

In [ ]:
plt.hist(sorted_metro_data.MedianRent)


(array([ 30.,  77.,  79.,  37.,   8.,  10.,   3.,   6.,   0.,   1.]),
 array([  400.,   540.,   680.,   820.,   960.,  1100.,  1240.,  1380.,
         1520.,  1660.,  1800.]),
 <a list of 10 Patch objects>)

In [ ]:
bootstrap_model_results = []
for i in range(0, 1000):
    array2 = nsmd.as_matrix(['PercentCollege', 'Percent18to35'])
    #array2 = nsmd
    
    test_labels = [-1, -1, -1, 1, 1, 1, 1, 1, 1]
    rng.shuffle(test_labels)

    end_labels = [-1] * (array2.shape[0] - 15) + [0] * 6
    rng.shuffle(end_labels)


    labels2 = np.asarray(test_labels + end_labels)

    prop2 = label_propagation.LabelPropagation(kernel='knn', n_neighbors=7, max_iter=100, alpha=1).fit(array2, labels2)
    model_results = prop2.predict(array2)
    #np.where(np.asarray(prop2[1]) > 0)
    bootstrap_model_results.append(model_results)

probabilities = np.array([0 for x in range(0, len(bootstrap_model_results[1]))])
for vec in bootstrap_model_results:
    probabilities = np.add(vec / 1000, probabilities)
probabilities
    

In [ ]:
plt.hist(probabilities, bins=25)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA

#similarities2 = euclidean_distances(nsmd)
similarities2 = euclidean_distances(array2)

seed = np.random.RandomState(seed=3)
mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, random_state=seed,
                   dissimilarity="precomputed", n_jobs=1)
pos = mds.fit(similarities2).embedding_
nmds = manifold.MDS(n_components=2, metric=False, max_iter=3000, eps=1e-12,
                    dissimilarity="precomputed", random_state=seed, n_jobs=1,
                    n_init=1)
npos = nmds.fit_transform(similarities2, init=pos)

In [ ]:
%matplotlib inline
# Rescale the data
pos *= np.sqrt((array2 ** 2).sum()) / np.sqrt((pos ** 2).sum())
npos *= np.sqrt((array2 ** 2).sum()) / np.sqrt((npos ** 2).sum())

# Rotate the data
clf = PCA(n_components=2)
array2 = clf.fit_transform(array2)

pos = clf.fit_transform(pos)

npos = clf.fit_transform(npos)

fig = plt.figure(1)
ax = plt.axes([0., 0., 1., 1.])

s = 50
plt.scatter(npos[np.where(np.asarray(probabilities) >= .5), 0], npos[np.where(np.asarray(probabilities) >= .5), 1], color='blue', s=s, lw=0, label='Good for tech')
plt.scatter(npos[np.where(np.asarray(probabilities) < .5), 0], npos[np.where(np.asarray(probabilities) < .5), 1], color='darkorange', s=s, lw=0, label='Less good for tech')

plt.legend(scatterpoints=1, loc='best', shadow=False)

In [ ]:
indices = np.where(np.asarray(probabilities) >= 0.5)
#sorted_metro_data.iloc[indices]['CBSA']

indices

In [ ]:
indices = np.where(np.asarray(probabilities) >= 0.5)
good_for_tech = sorted_metro_data.iloc[indices]

In [ ]:
similarity_matrix = euclidean_distances(nsmd)

sorted_metro_data['GoodForTech'] = np.where(np.asarray(probabilities) >= 0.5, 'Yes', 'No')

In [ ]:
sorted_metro_data

In [ ]:
import pymysql.cursors

user = 'kkutchko'
passwd = 'password'
database = 'metro_areas'

connection = pymysql.connect(host='localhost',
    user=user,
    password=passwd,
    db=database,
    cursorclass=pymysql.cursors.DictCursor)

drop_table_cmd = "drop table if exists metro_data"
drop_table_cmd2 = "drop table if exists city_distance"

create_table_cmd = "create table if not exists metro_data (" +\
    "CBSA varchar(64), " +\
    "TotalPeople int(8), " +\
    "PercentCollege decimal(7, 4), " +\
    "Percent18to35 decimal(7, 4), " +\
    "MedianRent int(5), " +\
    "DiversityIndex decimal(5, 4), " +\
    "GoodForTech varchar(5), " +\
    "primary key (CBSA))"
    
create_table_cmd2 = "create table if not exists city_distance (" +\
    "CBSA1 varchar(64), " +\
    "CBSA2 varchar(64), " +\
    "distance decimal(11, 8), " +\
    "primary key (CBSA1, CBSA2))"

insert_cmd = "insert into metro_data " +\
    "(CBSA, TotalPeople, PercentCollege, Percent18to35, MedianRent, DiversityIndex, GoodForTech) " +\
    'values ("%s", %s, %s, %s, %s, %s, "%s") ' +\
    "on duplicate key update CBSA = CBSA"
    
insert_cmd2 = "insert into city_distance " +\
    "(CBSA1, CBSA2, distance) " +\
    'values ("%s", "%s", %s) ' +\
    'on duplicate key update CBSA1=CBSA1, CBSA2=CBSA2'
    
try:
    with connection.cursor() as cursor:
        # create table with city data
        cursor.execute(drop_table_cmd)
        cursor.execute(create_table_cmd)

        for idx, line in sorted_metro_data.iterrows():
            cursor.execute(insert_cmd, (line['CBSA'], line['TotalPeople'],
                line['PercentCollege'] * 100, line['Percent18to35'] * 100,
                line['MedianRent'], line['DiversityIndex'],
                line['GoodForTech']))
            
        # create table for euclidean distances
        cursor.execute(drop_table_cmd2)
        cursor.execute(create_table_cmd2)
        
        for x in range(0, len(similarity_matrix)):
            for y in range(0, len(similarity_matrix[x])):
                matrix_data = (sorted_metro_data['CBSA'].iloc[x],
                     sorted_metro_data['CBSA'].iloc[y],
                     float(similarity_matrix[x][y]))
                if matrix_data[0] != matrix_data[1]:
                    cursor.execute(insert_cmd2, matrix_data)
        
finally:
    connection.commit()
    connection.close()
    



In [ ]:
plt.imshow(similarity_matrix)